<div style="direction:ltr;line-height:300%;">
	<font face="Arial" size=5>
		<div align=center>
				<p></p>
				<p></p>
In the Name of God
            <p></p>
<br>
            Sharif University of Technology
            <br>
Computer Engineering Department
            <p></p>
Artificial Intelligence Course
            <br />
			<br />
            MohammadHossein Rohban
            <br />
Fall 2021
        </div>
		<hr/>
			<div align=center>
Generative Adversarial Networks
        </div>
		<br />
		<div align=center>
Arman Zarei
        </div>
		<hr />
		<style type="text/css" scoped>
        p{
        border: 1px solid #a2a9b1;background-color: #f8f9fa;display: inline-block;
        };
        </style>
		<div>
			<h3>Table of Contents</h3>
			<ul style="margin-right: 0;">
				<li>
                    <a href="#sec_intro">
                        Introduction
                    </a>
                </li>
                <li>
					<a href="#sec_1">
                    Section 1
                    </a>
				</li>
                <li>
					<a href="#sec_2">
                Section 2
                    </a>
				</li>
			</ul>
		</div>
	</font>
</div>

<p></p>
<br />
<div id="sec_intro" style="direction:ltr;line-height:300%;">
	<font face="Arial" size=5>
		<font color=#888888 size=6>
Introduction
        </font>
		<p></p>
		<hr>
GANs are a framework for teaching a DL model to capture the training data’s distribution so we can generate new data from that same distribution. GANs were invented by Ian Goodfellow in 2014 and first described in the paper Generative Adversarial Nets. They are made of two distinct models, a <strong>generator</strong> and a <strong>discriminator</strong>.
<ul>
	<li>The job of the generator is to spawn ‘fake’ images that look like the training images.</li>
	<li>The job of the discriminator is to look at an image and output whether or not it is a real training image or a fake image from the generator.</li>
</ul>
   During training, 
	<ul>
		<li>the generator is constantly trying to outsmart the discriminator by generating better and better fakes,</li>
		<li>while the discriminator is working to become a better detective and correctly classify the real and fake images.</li>
	</ul>
     The equilibrium of this game is when the generator is generating perfect fakes that look as if they came directly from the training data, and the discriminator is left to always guess at 50% confidence that the generator output is real or fake.
        <br>
        <br/>
		<div style='text-align: center;'>
			<img src='./images/GAN.PNG'>
		</div>

<p></p>
<br />
<div id="sec_1" style="direction:ltr;line-height:300%;">
	<font face="Arial" size=5>
		<font color=#888888 size=6>
Theory of GAN
        </font>
		<p></p>
		<hr>

First, Lets define some notations:
- $x$: Data representing an image
- $D(x)$: The discriminator network which outputs the (scalar) probability that $x$ came from training data rather than the generator
    - $D(x)$ should be **high** when $x$ comes from training data
    - and **low** when $x$ comes from the generator.
- $z$: A latent space vector sampled from a standard normal distribution 
- $G(z)$: The generator function which maps the latent vector $z$ to data-space. 
    - goal of $G$: estimate the distribution that the training data comes from ($p_{data}$) so it can generate fake samples from that estimated distribution ($p_g$).

So, $D(G(z))$ is the probability (scalar) that the output of the generator GG is a real image. As described in Goodfellow’s paper, $D$ and $G$ play a minimax game in which DD tries to maximize the probability it correctly classifies reals and fakes ($logD(x)$), and $G$ tries to minimize the probability that $D$ will predict its outputs are fake ($log(1-D(G(z)))$).  From the paper, the GAN loss function is:
$$
\min_{G}\max_{D}V(D, G) = \mathbb{E}_{x \sim p_{data}(x)} [logD(x)] + \mathbb{E}_{z \sim p_{z}(z)} [log(1 - D(G(z)))]
$$
In theory, the solution to this minimax game is where $p_g=p_{data}$, and the discriminator guesses randomly if the inputs are real or fake. However, the convergence theory of GANs is still being actively researched and in reality models do not always train to this point.

<p></p>
<br />
<div id="sec_2" style="direction:ltr;line-height:300%;">
	<font face="Arial" size=5>
		<font color=#888888 size=6>
Implementation of GAN in PyTorch
        </font>
		<p></p>
		<hr>
		

In [ ]:
# Code Here

<p></p>
<br />
<div id="sec_2" style="direction:ltr;line-height:300%;">
	<font face="Arial" size=5>
		<font color=#888888 size=6>
Conditional GAN
        </font>
		<p></p>
		<hr>
        The conditional generative adversarial network, or cGAN for short, is a type of GAN that involves the conditional generation of images by a generator model. Image generation can be conditional on a class label, if available, allowing the targeted generated of images of a given type.
        <ul>
            <li>
                For example, imagine the MNIST dataset and suppose we want to generate samples of numbers, but, we want to control which number to generate.
            </li>
        </ul>
        Here is a comparison of GANs and cGANs:
        <br>
        <div style="text-align: center;">
            <img src='https://learnopencv.com/wp-content/uploads/2021/07/Conditional-GAN-in-PyTorch-and-TensorFlow.jpeg'>
        </div>
		

<p></p>
<br />
<div id="sec_2" style="direction:ltr;line-height:300%;">
	<font face="Arial" size=5>
		<font color=#888888 size=6>
Implementation of cGAN in PyTorch
        </font>
		<p></p>
		<hr>


In [ ]:
# Code Here

<p></p>
<br />
<div id="sec_2" style="direction:ltr;line-height:300%;">
	<font face="Arial" size=5>
		<font color=#888888 size=6>
Other types of GAN
        </font>
		<p></p>
		<hr>
		Some information about other variations of GAN ... (+ some links)

<p></p>
<br/>
<div id="sec_refs" style="direction:ltr;line-height:300%;">
	<font face="Arial" size=5>
		<font color=#888888 size=6>
References
        </font>
		<hr>       
        <ul>
            <li>
Reference 1
            </li>
            <li>
Reference 2
            </li>
        </ul>
	</font>
</div>